### <center> VOTING RULES IN PYTHON </center>
#### <center> MASTER BDMA - DECISION MODELING </center>
##### <center> Authors: Dilbar Isakova, MD Kamrul Islam </center>
##### <center> Professor: Brice Mayag </center>
###### <center> CentraleSupeléc, Fall, 2024 </center>

## Problem Statement

Let us consider an election with *n* voters (0 ≤ n ≤ 200) and *m* candidates (0 ≤ m ≤ 20). We assume that:

- The preferences of each voter are given as a linear order (total order) on the set of candidates.
- All the preferences (of the *n* voters) are contained in an Excel file or a csv file.

This work aims at computing in python language the voting rules introduced in Chapter 2. You can use the examples of this chapter to test your functionalities, especially the following example where we have *m = 4* candidates {a, b, c, d} and *n = 27* voters:

| Number of Voters | Preference Order           |
|------------------|----------------------------|
| 5 voters         | a > b > c > d              |
| 4 voters         | a > c > b > d              |
| 2 voters         | d > b > a > c              |
| 6 voters         | d > b > c > a              |
| 8 voters         | c > b > a > d              |
| 2 voters         | d > c > b > a              |

For the questions 2, 3, 4, and 5, you will have to manage yourself any possible ties among the winners.


#### Importing necessary libraries 

In [1]:
import csv
import random
import pandas as pd
from collections import Counter

#### Generate the data in csv format and read it for further analysis 

In [2]:
def generate_voting_csv(file_path):
    voting_data = [
        (5, ['a', 'b', 'c', 'd']),
        (4, ['a', 'c', 'b', 'd']),
        (2, ['d', 'b', 'a', 'c']),
        (6, ['d', 'b', 'c', 'a']),
        (8, ['c', 'b', 'a', 'd']),
        (2, ['d', 'c', 'b', 'a'])
    ]
    
    with open(file_path, mode='w', newline='') as file:
        csv_writer = csv.writer(file)

        csv_writer.writerow(['Voter Count', '1st Choice', '2nd Choice', '3rd Choice', '4th Choice'])

        for num_voters, preference in voting_data:
            csv_writer.writerow([num_voters] + preference)


file_path = 'voter_preferences.csv'
generate_voting_csv(file_path)


def read_voting_data(file_path):
    voting_data = []
    
    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)
        for row in csv_reader:
            if row:
                num_voters = int(row[0])
                preferences = row[1:]
                voting_data.append((num_voters, preferences))
    
    return voting_data

### Question 01
1. Compute a function Plurality returning the result of a plurality voting.

In [3]:
def plurality_voting(file_path):
    """
    Implements the plurality voting rule using the voting data format.
    The candidate with the most first-choice votes is the winner.
    """
    voting_data = read_voting_data(file_path)
    
    first_choice_votes = Counter()
    for num_voters, preferences in voting_data:
        first_choice = preferences[0] 
        first_choice_votes[first_choice] += num_voters
    
    winner = max(first_choice_votes, key=first_choice_votes.get)
    
    return winner, dict(first_choice_votes)

winner, vote_count = plurality_voting(file_path)

print(f"Winner of the election (Plurality): {winner}")
print("Vote Tally:")
for candidate, votes in vote_count.items():
    print(f"{candidate}: {votes} votes")

Winner of the election (Plurality): d
Vote Tally:
a: 9 votes
d: 10 votes
c: 8 votes


### Question 02
2. Compute a function PluralityRunoff returning the result of a plurality Runoff voting (plurality with two
rounds).

In [4]:
def plurality_runoff_voting(file_path):
    """
    Implements the plurality runoff voting rule using the optimized voting data format.
    The candidate with the most first-choice votes wins if they get more than 50% of the votes in the first round.
    If no candidate has more than 50%, a runoff is held between the top two candidates.
    """
    voting_data = read_voting_data(file_path)

    first_choice_votes = Counter()
    total_votes = 0
    
    # Tally the first-choice votes based on the number of voters and their preferences
    for num_voters, preferences in voting_data:
        first_choice = preferences[0]
        first_choice_votes[first_choice] += num_voters
        total_votes += num_voters
    
    # Check if any candidate has more than 50% of the total votes in the first round
    for candidate, votes in first_choice_votes.items():
        if votes > total_votes / 2:
            return candidate, dict(first_choice_votes),
    
    # If no one wins in the first round, we proceed to a runoff between the top two candidates
    top_two_candidates = first_choice_votes.most_common(2)
    candidate_1, candidate_2 = top_two_candidates[0][0], top_two_candidates[1][0]
    
    runoff_votes = {candidate_1: 0, candidate_2: 0}
    
    # Count votes for the top two candidates in the second round based on preferences
    for num_voters, preferences in voting_data:
        if preferences[0] == candidate_1 or preferences[0] == candidate_2:
            # If the first choice is one of the top two candidates, keep it
            runoff_votes[preferences[0]] += num_voters
        else:
            # If the first choice is not in the top two, use the highest-ranked candidate from the two
            for preference in preferences:
                if preference == candidate_1 or preference == candidate_2:
                    runoff_votes[preference] += num_voters
                    break
    
    runoff_winner = max(runoff_votes, key=runoff_votes.get)
    
    return runoff_winner, dict(first_choice_votes), dict(runoff_votes)


runoff_winner, first_round_results, second_round_results = plurality_runoff_voting(file_path)

print(f"Winner of the election (Plurality Runoff): {runoff_winner}")
print("First Round Vote Tally:")
for candidate, votes in first_round_results.items():
    print(f"{candidate}: {votes} votes")

if second_round_results:
    print("\nRunoff Round Vote Tally:")
    for candidate, votes in second_round_results.items():
        print(f"{candidate}: {votes} votes")


Winner of the election (Plurality Runoff): a
First Round Vote Tally:
a: 9 votes
d: 10 votes
c: 8 votes

Runoff Round Vote Tally:
d: 10 votes
a: 17 votes


### Question 03
3. Compute a function CondorcetVoting returning the result of the application of the Condorcet principle (the existence of the Condorcet winner).

In [5]:
def condorcet_voting(file_path):
    """
    Implements the Condorcet voting rule using the optimized voting data format.
    A Condorcet winner is a candidate who wins against every other candidate in pairwise comparisons.
    """
    voting_data = read_voting_data(file_path)
    
    # Get the list of all candidates
    all_candidates = set()
    for _, preferences in voting_data:
        all_candidates.update(preferences)
    
    all_candidates = list(all_candidates)
    
    # Initialize pairwise comparison matrix
    pairwise_comparisons = {candidate: {other: 0 for other in all_candidates if other != candidate} for candidate in all_candidates}
    
    # Conduct pairwise comparisons
    for num_voters, preferences in voting_data:
        for i, candidate in enumerate(preferences):
            for other in preferences[i + 1:]:
                pairwise_comparisons[candidate][other] += num_voters
    
    # Check if there is a Condorcet winner
    condorcet_winner = None
    for candidate in all_candidates:
        is_winner = True
        for other in all_candidates:
            if candidate != other and pairwise_comparisons[candidate][other] <= pairwise_comparisons[other][candidate]:
                is_winner = False
                break
        if is_winner:
            condorcet_winner = candidate
            break
    
    return condorcet_winner, pairwise_comparisons

condorcet_winner, pairwise_results = condorcet_voting(file_path)

if condorcet_winner:
    print(f"Condorcet Winner: {condorcet_winner}")
else:
    print("No Condorcet Winner")

print("\nPairwise Comparison Results:")
for candidate, comparisons in pairwise_results.items():
    for other, votes in comparisons.items():
        print(f"{candidate} vs {other}: {votes} votes")

Condorcet Winner: c

Pairwise Comparison Results:
c vs d: 17 votes
c vs a: 16 votes
c vs b: 14 votes
d vs c: 10 votes
d vs a: 10 votes
d vs b: 10 votes
a vs c: 11 votes
a vs d: 17 votes
a vs b: 9 votes
b vs c: 13 votes
b vs d: 17 votes
b vs a: 18 votes


### Question 04
4. Compute a function BordaVoting returning the result of the application of the Borda principle.

In [6]:
def borda_voting(file_path):
    """
    Implements the Borda voting rule using the optimized voting data format.
    Each candidate is assigned points based on their rank in each voter's preference. 
    The candidate with the highest total score is the winner.
    """
    
    voting_data = read_voting_data(file_path)
    
    # Get the list of all candidates
    all_candidates = set()
    for _, preferences in voting_data:
        all_candidates.update(preferences)
    
    all_candidates = list(all_candidates)
    
    # Initialize Borda scores for each candidate
    borda_scores = {candidate: 0 for candidate in all_candidates}
    
    # Assign points based on rank
    num_candidates = len(all_candidates)
    
    for num_voters, preferences in voting_data:
        for i, candidate in enumerate(preferences):
            # n-1 points for first, n-2 for second, ..., 0 for last
            borda_scores[candidate] += num_voters * (num_candidates - 1 - i)
    
    # Find the candidate with the highest Borda score
    winner = max(borda_scores, key=borda_scores.get)
    
    return winner, borda_scores


borda_winner, borda_scores = borda_voting(file_path)

print(f"Winner of the election (Borda Voting): {borda_winner}")
print("Borda Scores:")
for candidate, score in borda_scores.items():
    print(f"{candidate}: {score} points")

Winner of the election (Borda Voting): b
Borda Scores:
c: 47 points
d: 30 points
a: 37 points
b: 48 points


### Question 05
5. Elaborate an election example with n ≥ 60 and m ≥ 8 where the winner is the same for the four voting rules Plurality, Plurality with Runoff, Condorcet Principle and Borda rules. 

In your example, at least 20% of voters should have different preferences and no more than 70% of voters has the same “best candidate”. You should implement, separately, a python function allowing to test if these two conditions are satisfied.

In [7]:
def generate_voter_preferences(n_voters, candidates):
    preferences = []
    for _ in range(n_voters):
        preference = random.sample(candidates, len(candidates))
        preferences.append(tuple(preference))
    return preferences

def check_minimum_different_preferences(preferences, min_percentage=0.20):
    unique_preferences = set(preferences)
    return len(unique_preferences) >= len(preferences) * min_percentage

def check_max_same_best_candidate(preferences, max_percentage=0.70):
    first_choices = [p[0] for p in preferences]
    first_choice_counts = Counter(first_choices)
    max_first_choice_count = max(first_choice_counts.values())
    return max_first_choice_count <= len(preferences) * max_percentage

def plurality_voting(preferences):
    first_choice_votes = Counter(p[0] for p in preferences)
    winner = max(first_choice_votes, key=first_choice_votes.get)
    return winner, dict(first_choice_votes)

def plurality_runoff_voting(preferences):
    first_choice_votes = Counter(p[0] for p in preferences)
    total_votes = sum(first_choice_votes.values())

    for candidate, votes in first_choice_votes.items():
        if votes > total_votes / 2:
            return candidate, dict(first_choice_votes), None

    top_two = [candidate for candidate, _ in first_choice_votes.most_common(2)]
    runoff_votes = {top_two[0]: 0, top_two[1]: 0}

    for preference in preferences:
        for candidate in preference:
            if candidate in top_two:
                runoff_votes[candidate] += 1
                break

    runoff_winner = max(runoff_votes, key=runoff_votes.get)
    return runoff_winner, dict(first_choice_votes), runoff_votes

def condorcet_voting(preferences):
    candidates = set(c for preference in preferences for c in preference)
    pairwise_wins = {candidate: {other: 0 for other in candidates if other != candidate} for candidate in candidates}

    for preference in preferences:
        for i, candidate in enumerate(preference):
            for opponent in preference[i + 1:]:
                pairwise_wins[candidate][opponent] += 1

    for candidate in candidates:
        if all(pairwise_wins[candidate][other] > pairwise_wins[other][candidate] for other in candidates if other != candidate):
            return candidate, pairwise_wins
    return None, pairwise_wins

def borda_voting(preferences):
    candidates = set(c for preference in preferences for c in preference)
    borda_scores = {candidate: 0 for candidate in candidates}
    num_candidates = len(candidates)

    for preference in preferences:
        for rank, candidate in enumerate(preference):
            borda_scores[candidate] += num_candidates - rank - 1

    winner = max(borda_scores, key=borda_scores.get)
    return winner, borda_scores

def validate_data(preferences, candidates):
    results = {}
    results['Plurality'] = plurality_voting(preferences)
    results['Plurality Runoff'] = plurality_runoff_voting(preferences)
    results['Condorcet'] = condorcet_voting(preferences)
    results['Borda'] = borda_voting(preferences)

    same_winner = all(results[method][0] == results['Plurality'][0] for method in results if results[method][0] is not None)

    return results, same_winner

n_voters = 60
candidates = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

saved_preferences = None

for attempt in range(10000):
    preferences = generate_voter_preferences(n_voters, candidates)

    if check_minimum_different_preferences(preferences) and check_max_same_best_candidate(preferences):
        results, is_consistent = validate_data(preferences, candidates)

        if is_consistent and results['Condorcet'][0] is not None:
            saved_preferences = preferences 
            
            print(f"\nElection Results (Consistent winner found on attempt {attempt + 1}):\n")
            
            # Displaying concise results for each voting method
            print("1. Plurality Voting")
            plurality_winner, plurality_votes = results['Plurality']
            print(f"   Winner: {plurality_winner}")

            print("2. Plurality Runoff Voting")
            runoff_winner, first_round_votes, runoff_round_votes = results['Plurality Runoff']
            print(f"   Winner: {runoff_winner}")

            print("3. Condorcet Voting")
            condorcet_winner, _ = results['Condorcet']
            if condorcet_winner:
                print(f"   Winner: {condorcet_winner}")
            else:
                print("   No Condorcet Winner")
            print()

            print("4. Borda Voting")
            borda_winner, borda_scores = results['Borda']
            print(f"   Winner: {borda_winner}")

            # Displaying the voter preferences in DataFrame format
            print("\nSaved Preferences (Voter Rankings):")
            df_preferences = pd.DataFrame(saved_preferences, columns=[f'Rank {i+1}' for i in range(len(candidates))])
            print(df_preferences)
            break
else:
    print("Could not find a suitable election within 10,000 attempts.")



Election Results (Consistent winner found on attempt 1):

1. Plurality Voting
   Winner: g
2. Plurality Runoff Voting
   Winner: g
3. Condorcet Voting
   Winner: g

4. Borda Voting
   Winner: g

Saved Preferences (Voter Rankings):
   Rank 1 Rank 2 Rank 3 Rank 4 Rank 5 Rank 6 Rank 7 Rank 8
0       d      b      c      f      e      g      a      h
1       g      b      d      h      e      f      a      c
2       c      e      f      g      b      h      a      d
3       b      g      a      f      c      e      h      d
4       d      c      b      h      f      g      e      a
5       d      g      h      f      e      b      a      c
6       g      b      e      c      a      d      f      h
7       f      e      d      h      c      g      a      b
8       d      e      h      g      f      a      c      b
9       e      f      h      a      b      c      g      d
10      h      b      c      e      d      a      f      g
11      b      a      d      f      g      c      e      h
1

### Question 06

6. Is it possible to elaborate an election example with n ≥ 60 and m ≥ 8 where the unique winner is not the same for the four voting rules Plurality, Plurality with Runoff, Condorcet Principle and Borda rules (we should have 4 different winners) ?

In your example, at least 20% of voters should have different preferences and no more than 70% of voters has the
same “best candidate”. In order to test if these two conditions are satisfied, you can use the function implemented
above, in question 5.

In [8]:
def generate_voter_preferences(n_voters, candidates):
    """
    Generate random voter preferences. Each voter's preference is a random permutation of candidates.
    """
    preferences = []
    for _ in range(n_voters):
        preference = random.sample(candidates, len(candidates))
        preferences.append(tuple(preference)) 
    return preferences

def check_minimum_different_preferences(preferences, min_percentage=0.20):
    unique_preferences = set(preferences)
    return len(unique_preferences) >= len(preferences) * min_percentage

def check_max_same_best_candidate(preferences, max_percentage=0.70):
    first_choices = [p[0] for p in preferences]
    first_choice_counts = Counter(first_choices)
    max_first_choice_count = max(first_choice_counts.values())
    return max_first_choice_count <= len(preferences) * max_percentage


def plurality_voting(preferences):
    first_choice_votes = Counter(p[0] for p in preferences)
    winner = max(first_choice_votes, key=first_choice_votes.get)
    return winner

def plurality_runoff_voting(preferences):
    first_choice_votes = Counter(p[0] for p in preferences)
    total_votes = sum(first_choice_votes.values())

    for candidate, votes in first_choice_votes.items():
        if votes > total_votes / 2:
            return candidate
    

    top_two = [candidate for candidate, _ in first_choice_votes.most_common(2)]
    runoff_votes = {top_two[0]: 0, top_two[1]: 0}
    
    for preference in preferences:
        for candidate in preference:
            if candidate in top_two:
                runoff_votes[candidate] += 1
                break
    
    return max(runoff_votes, key=runoff_votes.get)

def condorcet_voting(preferences, candidates):
    pairwise_wins = {candidate: {other: 0 for other in candidates if other != candidate} for candidate in candidates}
    
    for preference in preferences:
        for i, candidate in enumerate(preference):
            for opponent in preference[i + 1:]:
                pairwise_wins[candidate][opponent] += 1

    for candidate in candidates:
        if all(pairwise_wins[candidate][other] > pairwise_wins[other][candidate] for other in candidates if other != candidate):
            return candidate
    return None

def borda_voting(preferences, candidates):
    borda_scores = {candidate: 0 for candidate in candidates}
    num_candidates = len(candidates)

    for preference in preferences:
        for rank, candidate in enumerate(preference):
            borda_scores[candidate] += num_candidates - rank - 1

    return max(borda_scores, key=borda_scores.get)

def validate_unique_winners(preferences, candidates):
    """
    Determine if each voting method produces a different winner.
    """
    results = {
        'Plurality': plurality_voting(preferences),
        'Plurality Runoff': plurality_runoff_voting(preferences),
        'Condorcet': condorcet_voting(preferences, candidates),
        'Borda': borda_voting(preferences, candidates)
    }
    
    unique_winners = len(set(results.values())) == 4
    return results, unique_winners

def find_unique_election_data(n_voters=60, candidates=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'], max_attempts=10000):
    for attempt in range(max_attempts):
        preferences = generate_voter_preferences(n_voters, candidates)

        if check_minimum_different_preferences(preferences) and check_max_same_best_candidate(preferences):
            results, unique_winners = validate_unique_winners(preferences, candidates)
            
            if unique_winners:
                print(f"\nUnique Election Results (Found on attempt {attempt + 1}):\n")
                for method, winner in results.items():
                    print(f"{method} Winner: {winner}")
                
                df_unique_preferences = pd.DataFrame(preferences, columns=[f'Rank {i+1}' for i in range(len(candidates))])
                print("\nVoter Preferences (Sample):")
                print(df_unique_preferences)
                return df_unique_preferences

    print(f"\nCould not find unique winners in {max_attempts} attempts.")
    return None

stored_unique_results = find_unique_election_data()


Unique Election Results (Found on attempt 10):

Plurality Winner: b
Plurality Runoff Winner: a
Condorcet Winner: None
Borda Winner: f

Voter Preferences (Sample):
   Rank 1 Rank 2 Rank 3 Rank 4 Rank 5 Rank 6 Rank 7 Rank 8
0       b      f      c      g      d      h      e      a
1       f      a      b      d      h      g      c      e
2       b      a      g      e      c      d      h      f
3       h      b      f      d      a      g      e      c
4       h      c      f      g      a      d      e      b
5       c      f      a      g      e      h      d      b
6       a      b      h      g      e      d      f      c
7       b      a      e      f      h      d      c      g
8       b      c      h      a      g      f      d      e
9       f      e      h      d      g      a      c      b
10      a      h      f      c      b      d      e      g
11      g      e      d      c      a      f      b      h
12      c      f      a      d      g      b      h      e
13      b 